In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
df=df.read_csv("")

In [ ]:

df=pd.DataFrame(boston.data)
df.columns=boston.feature_names
df["Price"]=boston.target
